<a href="https://colab.research.google.com/github/BrandonUGS/ModeloCovid2023/blob/main/Proyecto_Final_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

fatal: destination path 'rapidsai-csp-utils' already exists and is not an empty directory.
Traceback (most recent call last):
  File "/content/rapidsai-csp-utils/colab/pip-install.py", line 28, in <module>
    if ('K80' not in gpu_name):
TypeError: a bytes-like object is required, not 'str'


In [ ]:
!wget https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip

--2023-11-12 19:43:16--  https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip
Resolving datosabiertos.salud.gob.mx (datosabiertos.salud.gob.mx)... 201.98.60.146
Connecting to datosabiertos.salud.gob.mx (datosabiertos.salud.gob.mx)|201.98.60.146|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18045440 (17M) [application/x-zip-compressed]
Saving to: ‘datos_abiertos_covid19.zip.2’

datos_abiertos_covi 100%[===================>]  17.21M  7.25MB/s    in 2.4s    

2023-11-12 19:43:19 (7.25 MB/s) - ‘datos_abiertos_covid19.zip.2’ saved [18045440/18045440]



In [ ]:
!unzip /content/datos_abiertos_covid19.zip

Archive:  /content/datos_abiertos_covid19.zip
  inflating: COVID19MEXICO.csv       


In [ ]:
import numpy as np
import pandas as pd


data = pd.read_csv('COVID19MEXICO.csv')
data.columns

Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'INDIGENA',
       'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'TOMA_MUESTRA_LAB', 'RESULTADO_LAB',
       'TOMA_MUESTRA_ANTIGENO', 'RESULTADO_ANTIGENO', 'CLASIFICACION_FINAL',
       'MIGRANTE', 'PAIS_NACIONALIDAD', 'PAIS_ORIGEN', 'UCI'],
      dtype='object')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1186694 entries, 0 to 1186693
Data columns (total 40 columns):
 #   Column                 Non-Null Count    Dtype 
---  ------                 --------------    ----- 
 0   FECHA_ACTUALIZACION    1186694 non-null  object
 1   ID_REGISTRO            1186694 non-null  object
 2   ORIGEN                 1186694 non-null  int64 
 3   SECTOR                 1186694 non-null  int64 
 4   ENTIDAD_UM             1186694 non-null  int64 
 5   SEXO                   1186694 non-null  int64 
 6   ENTIDAD_NAC            1186694 non-null  int64 
 7   ENTIDAD_RES            1186694 non-null  int64 
 8   MUNICIPIO_RES          1186694 non-null  int64 
 9   TIPO_PACIENTE          1186694 non-null  int64 
 10  FECHA_INGRESO          1186694 non-null  object
 11  FECHA_SINTOMAS         1186694 non-null  object
 12  FECHA_DEF              1186694 non-null  object
 13  INTUBADO               1186694 non-null  int64 
 14  NEUMONIA               1186694 non

In [ ]:
df = data

df = df.drop(columns=['SECTOR','MUNICIPIO_RES','HABLA_LENGUA_INDIG','ID_REGISTRO','ORIGEN','PAIS_NACIONALIDAD','PAIS_ORIGEN','INDIGENA','ENTIDAD_UM','ENTIDAD_NAC','NACIONALIDAD','ENTIDAD_RES'])
# Lista de clasificaciones que deseas eliminar
clasificaciones_a_eliminar = [2, 4, 5]

# Filtrar el DataFrame para excluir las clasificaciones no deseadas
df = df[~df['CLASIFICACION_FINAL'].isin(clasificaciones_a_eliminar)]

df.columns

df['FECHA_ACTUALIZACION'] = pd.to_datetime(df['FECHA_ACTUALIZACION'])
df['FECHA_INGRESO'] = pd.to_datetime(df['FECHA_INGRESO'])
df['FECHA_SINTOMAS'] = pd.to_datetime(df['FECHA_SINTOMAS'])
df['FECHA_DEF'] = pd.to_datetime(df['FECHA_DEF'], errors='coerce')


for col in ['TIPO_PACIENTE']:
  df[col]=df[col].astype(str)


from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in ['TIPO_PACIENTE','SEXO']:
  df[col]=le.fit_transform(df[col])

 #crear caracteristicas apartir de las fechas

df['DIAS_DEFUNCION']=(df['FECHA_ACTUALIZACION'] - df['FECHA_DEF']).dt.days
df['DIAS_HOSPITALIZACION']=(df['FECHA_ACTUALIZACION'] - df['FECHA_INGRESO']).dt.days
df['DIAS_SINTOMAS']=(df['FECHA_ACTUALIZACION'] - df['FECHA_SINTOMAS']).dt.days

# Red neuronal no sera compatible debido a los NAN sera necesario cambiarlo a 0
df['DIAS_DEFUNCION']=df['DIAS_DEFUNCION'].apply(lambda x: 0 if x < 0 else x)

df = df.drop(columns=['FECHA_ACTUALIZACION', 'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF','DIAS_DEFUNCION','MIGRANTE'])

X = df.drop(columns=['CLASIFICACION_FINAL'])
y = df['CLASIFICACION_FINAL']



<ipython-input-13-256143fdde0c>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['FECHA_ACTUALIZACION'] = pd.to_datetime(df['FECHA_ACTUALIZACION'])


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 42)



In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1184327 entries, 0 to 1186693
Data columns (total 26 columns):
 #   Column                 Non-Null Count    Dtype
---  ------                 --------------    -----
 0   SEXO                   1184327 non-null  int64
 1   TIPO_PACIENTE          1184327 non-null  int64
 2   INTUBADO               1184327 non-null  int64
 3   NEUMONIA               1184327 non-null  int64
 4   EDAD                   1184327 non-null  int64
 5   EMBARAZO               1184327 non-null  int64
 6   DIABETES               1184327 non-null  int64
 7   EPOC                   1184327 non-null  int64
 8   ASMA                   1184327 non-null  int64
 9   INMUSUPR               1184327 non-null  int64
 10  HIPERTENSION           1184327 non-null  int64
 11  OTRA_COM               1184327 non-null  int64
 12  CARDIOVASCULAR         1184327 non-null  int64
 13  OBESIDAD               1184327 non-null  int64
 14  RENAL_CRONICA          1184327 non-null  int64
 15

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report


clf = RandomForestClassifier(random_state=42)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           1       0.14      0.03      0.05      1579
           3       1.00      1.00      1.00     83925
           6       0.97      0.97      0.97     13017
           7       0.99      1.00      0.99    138345

    accuracy                           0.99    236866
   macro avg       0.77      0.75      0.75    236866
weighted avg       0.99      0.99      0.99    236866



In [ ]:
import cuml

from cuml.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report


clf = cuml.RandomForestClassifier(random_state=42)
X_train= X_train.astype('float32')
y_train= y_train.astype('float32')

clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

/usr/local/lib/python3.10/dist-packages/cuml/internals/api_decorators.py:344: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams=1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  return func(**kwargs)


              precision    recall  f1-score   support

           1       0.72      0.01      0.02      1579
           3       1.00      1.00      1.00     83925
           6       0.97      0.97      0.97     13017
           7       0.99      1.00      0.99    138345

    accuracy                           0.99    236866
   macro avg       0.92      0.75      0.75    236866
weighted avg       0.99      0.99      0.99    236866



In [ ]:
import cuml

from cuml.ensemble import RandomForestClassifier
from cuml.neighbors import KNeighborsClassifier


cuml_RFC = cuml.RandomForestClassifier(random_state=42)
cuml_KNC = cuml.KNeighborsClassifier()


classfiers = [
    {
        'Random Forest',cuml_RFC
    },
    {
        'KNN',cuml_RFC
    }

]

for name,clf in classfiers :
  clf.fit(X_train,y_train)
  y_pred = clf.predict(X_test)
  print(classification_report(y_test, y_pred))

/usr/local/lib/python3.10/dist-packages/cuml/internals/api_decorators.py:344: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams=1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  return func(**kwargs)


AttributeError: ignored

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 14.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.7/302.7 kB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.8/395.8 kB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 65.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.4

In [ ]:
import gradio as gr
import pandas as pd
import numpy as np


def predict(sexo, tipo_paciente, intubado, neumonia, edad, embarazo,
             diabetes, epoc, asma, inmusupr,
            hipertension, otra_com, cardiovascular, obesidad,
            renal_cronica, tabaquismo, otro_caso, toma_muestra_lab,
            resultado_lab, toma_muestra_antigeno, resultado_antigeno,
           uci, dias_hospitalizacion, dias_sintomas):


    # Crea un DataFrame con los valores de entrada
    input_data = pd.DataFrame([[
        sexo, tipo_paciente, intubado, neumonia, edad, embarazo,
       diabetes, epoc, asma, inmusupr,
        hipertension, otra_com, cardiovascular, obesidad,
        renal_cronica, tabaquismo, otro_caso, toma_muestra_lab,
        resultado_lab, toma_muestra_antigeno, resultado_antigeno, uci, dias_hospitalizacion, dias_sintomas
        ]], columns=[
        'SEXO', 'TIPO_PACIENTE', 'INTUBADO', 'NEUMONIA', 'EDAD', 'EMBARAZO','DIABETES', 'EPOC',
        'ASMA', 'INMUSUPR','HIPERTENSION', 'OTRA_COM', 'CARDIOVASCULAR', 'OBESIDAD','RENAL_CRONICA',
        'TABAQUISMO', 'OTRO_CASO', 'TOMA_MUESTRA_LAB','RESULTADO_LAB', 'TOMA_MUESTRA_ANTIGENO',
        'RESULTADO_ANTIGENO','UCI', 'DIAS_HOSPITALIZACION', 'DIAS_SINTOMAS'])

    prediction = clf.predict(input_data)
    return prediction




gr.Row
inputs = [
    gr.Radio(choices=[0, 1], label='Sexo'),
    gr.Radio(choices=[0, 1], label='Tipo de Paciente'),
    gr.Radio(choices=[0, 1, 97, 98, 99], label='Intubado'),
    gr.Radio(choices=[0, 1, 97, 98, 99], label='Neumonía'),
    gr.Slider(minimum=0, maximum=120, value=30, label='Edad'),
    gr.Radio(choices=[1, 2, 97, 98, 99], label='Embarazo'),
    gr.Radio(choices=[1, 2, 98], label='Diabetes'),
    gr.Radio(choices=[1, 2, 98], label='EPOC'),
    gr.Radio(choices=[1, 2, 98], label='Asma'),
    gr.Radio(choices=[1, 2, 98], label='Inmunosupresión'),
    gr.Radio(choices=[1, 2, 98], label='Hipertensión'),
    gr.Radio(choices=[1, 2, 98], label='Otra Comorbilidad'),
    gr.Radio(choices=[1, 2, 98], label='Cardiovascular'),
    gr.Radio(choices=[1, 2, 98], label='Obesidad'),
    gr.Radio(choices=[1, 2, 98], label='Enfermedad Renal Crónica'),
    gr.Radio(choices=[1, 2, 98], label='Tabaquismo'),
    gr.Radio(choices=[1, 2, 99], label='Contacto con otro caso'),
    gr.Radio(choices=[1, 2, 98], label='Toma de muestra de laboratorio'),
    gr.Radio(choices=[1, 2, 97, 98, 99], label='Resultado de laboratorio'),
    gr.Radio(choices=[1, 2, 98], label='Toma de muestra de antígeno'),
    gr.Radio(choices=[1, 2, 97, 98, 99], label='Resultado de antígeno'),
    gr.Radio(choices=[1, 2, 97, 98, 99], label='UCI'),
    gr.Number(label='Días de Hospitalización',value=0),
    gr.Number(label='Días con Síntomas',value=0),
]

# Crear componente de salida para Gradio
outputs = gr.Textbox(label="Predicción")

# Crear la interfaz de Gradio
demo = gr.Interface(fn=predict, inputs=inputs, outputs=outputs)

# Ejecutar la aplicación web
if __name__ == "__main__":
    demo.launch(show_api=False,debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://4fb84e6801def0715d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://4fb84e6801def0715d.gradio.live
